## Projected Policy Gradient method (PPG) and Projected Policy Gradient with Interpoliation (PPGI) for the 2s2a toy example.

In [61]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def project_onto_simplex(probabilities):
    sorted_probs = -np.sort(-probabilities)
    cumsum_probs = np.cumsum(sorted_probs)
    rho = np.where(sorted_probs > (cumsum_probs - 1) / (np.arange(len(probabilities)) + 1))[0][-1]
    theta = (cumsum_probs[rho] - 1) / (rho + 1)
    projection = np.maximum(probabilities - theta, 0)
    return projection

#### 用矩阵形式求 discounted state occupancy measure, 这里简便起见将它乘上 1/(1 - gamma) ，并用它求出 V 和 Q

In [62]:
def compute_d(states, actions, policy, transition_probs, gamma):
    """
    Compute d_{s}^{pi}(s') for all s, s' using the direct matrix form of discounted visitation frequency.

    Args:
        states: List of states.
        actions: List of actions.
        policy: Current policy (shape: [num_states, num_actions]).
        transition_probs: Transition probabilities (shape: [num_states, num_actions, num_states]).
        gamma: Discount factor.

    Returns:
        d_s_pi: Discounted visitation frequencies (shape: [num_states, num_states]).
                d_s_pi[s, s'] represents d_{s}^{pi}(s').
    """
    num_states = len(states)
    num_actions = len(actions)

    # 构建策略下的转移矩阵 P_pi
    P_pi = np.zeros((num_states, num_states))
    for s in range(num_states):
        for a in range(num_actions):
            P_pi[s, :] += policy[s, a] * transition_probs[s, a, :]

    # 构建单位矩阵 I
    I = np.eye(num_states)

    # 计算 (I - gamma * P_pi) 矩阵
    A = I - gamma * P_pi

    # 检查矩阵是否可逆
    if np.linalg.matrix_rank(A) < num_states:
        raise ValueError("矩阵 (I - gamma * P_pi) 不可逆，可能需要调整折扣因子 gamma 或检查转移概率。")

    # 计算 D = (I - gamma * P_pi)^(-1)
    D = np.linalg.inv(A)

    return D


def compute_v_q_from_d(states, actions, D, policy, rewards, gamma):
    """
    Compute V and Q from d_{s}^{pi}(s').

    Args:
        states: List of states.
        actions: List of actions.
        D: Discounted visitation frequencies (shape: [num_states, num_states]).
        policy: Current policy (shape: [num_states, num_actions]).
        rewards: Reward function (shape: [num_states, num_actions]).
        gamma: Discount factor.

    Returns:
        V: Value function (shape: [num_states]).
        Q: Action-value function (shape: [num_states, num_actions]).
    """
    num_states = len(states)
    num_actions = len(actions)
    V = np.zeros(num_states)
    Q = np.zeros((num_states, num_actions))

    # Compute V(s) using D
    for s in range(num_states):
        V[s] = np.sum([
            D[s, s_prime] * np.sum([
                policy[s_prime, a] * rewards[s_prime, a]
                for a in range(num_actions)
            ])
            for s_prime in range(num_states)
        ])

    # Compute Q(s, a) using the Bellman equation
    for s in range(num_states):
        for a in range(num_actions):
            Q[s, a] = rewards[s, a] + gamma * np.sum([
                transition_probs[s, a, s_prime] * V[s_prime]
                for s_prime in range(num_states)
            ])

    return V, Q

In [63]:
def visualize_policy_joint_trajectory(policy_trajectory, eta, alpha):
    
    # Extract coordinates for π(s0, a0) and π(s1, a0)
    x_coords = [policy[0, 0] for policy in policy_trajectory]  # π(s0, a0)
    y_coords = [policy[1, 0] for policy in policy_trajectory]  # π(s1, a0)

    plt.figure(figsize=(8, 8))
    plt.title("Policy Trajectory for π(s0, a0) vs π(s1, a0)")

    # Plot the trajectory
    plt.plot(x_coords, y_coords, '-o', label="Policy Trajectory", alpha=0.8)

    # Mark the initial point with a different color and label
    plt.scatter(x_coords[0], y_coords[0], color='green', s=100, label="Initial Point")
    plt.text(x_coords[0], y_coords[0] + 0.02, "Initial", color='green', fontsize=10, ha='center')

    # Mark the final point with a different color and label
    plt.scatter(x_coords[-1], y_coords[-1], color='red', s=100, label="Final Point")
    plt.text(x_coords[-1], y_coords[-1] + 0.02, "Final", color='red', fontsize=10, ha='center')

    # Add labels for the axes
    plt.xlabel("π(s0, a0)")
    plt.ylabel("π(s1, a0)")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.grid(True)

    # Display the additional information about parameters a and b
    plt.text(0.5, 1.10, f"eta={eta}, alpha={alpha}", fontsize=12, ha='center', transform=plt.gca().transAxes)

    # Add a legend
    plt.legend()
    plt.show()

In [64]:
def projected_policy_gradient_interpolation(states, actions, transition_probs, rewards, gamma, rho, policy_init, eta, n_iter, alpha, tau):
    policy = policy_init.copy()
    policy_tar = policy_init.copy()  # Target policy for interpolation step
    V_history = []
    Q_history = []
    discounted_rewards = []  # Store the discounted rewards
    policy_trajectory = [policy.copy()]  # Track the trajectory of policies

    for k in range(n_iter):
        # Compute d_{s}^{pi}(s') using Monte Carlo
        d_s_pi = compute_d(states, actions, policy, transition_probs, gamma)

        # Compute V and Q using d_{s}^{pi}
        V, Q = compute_v_q_from_d(states, actions, d_s_pi, policy, rewards, gamma)
        V_history.append(V)
        Q_history.append(Q)

        # Compute total discounted reward
        discounted_reward = np.dot(rho, V)  # Use initial state distribution rho to compute reward
        discounted_rewards.append(discounted_reward)
        """
        # Output current values and policy
        print(f"Iteration {k}:")
        print("Value Function V:")
        print(V)
        print("Action-Value Function Q:")
        print(Q)
        print("Policy π:")
        print(policy)
        print("State Visitation Frequencies d_s_pi:")
        print(d_s_pi)
        """
        
        
        # Check termination conditions
        if k > 0:
            # Condition 1: Check if |V_{s0}^{k+1} - V_{s0}^k| < 1e-6
            if abs(V[0] - V_history[-2][0]) < 1e-8:
                # Condition 2: Check if |π^{k+1}(s, a) - π^{k}(s, a)| < 1e-6 for all (s, a)
                if np.all(np.abs(policy - policy_trajectory[-1]) < 1e-12):
                    print("Termination conditions met.")
                    break
        
        # Compute policy gradient
        gradient = np.zeros_like(policy)
        for s in range(len(states)):
            for a in range(len(actions)):
                gradient[s, a] = d_s_pi[0, s] * Q[s, a] + tau * np.divide(1, policy[s, a] + 1e-16) # initial state distribution rho = [1,0,0]
        
        # Policy update
        policy_tar = policy + eta * gradient
        
        # Projection onto simplex
        for s in range(len(states)):
            policy_tar[s] = project_onto_simplex(policy_tar[s])
            if np.isnan(policy[s]).any():
                raise ValueError(f"Policy contains NaN after projection at state {s}")
        
        # Interpolation step
        policy = (1 - alpha) * policy + alpha * policy_tar # alpha = 1 for no interpolation
        policy_trajectory.append(policy.copy())  # Track updated policy
    
    return V_history, Q_history, discounted_rewards, policy_trajectory


In [65]:
# 修改后的函数，支持多个初始策略
def save_policy_trajectories_to_pdf(output_pdf, eta_values, alpha_values, states, actions, transition_probs, rewards, gamma, rho, policy_inits, n_iter, tau):
    """
    Generate and save policy trajectories for different eta, alpha, and initial policy combinations to a PDF file.

    Args:
        output_pdf (str): The path to the output PDF file.
        eta_values (list): A list of eta values to iterate over.
        alpha_values (list): A list of alpha values to iterate over.
        states (list): List of states in the MDP.
        actions (list): List of actions in the MDP.
        transition_probs (ndarray): Transition probabilities (shape: [num_states, num_actions, num_states]).
        rewards (ndarray): Rewards matrix (shape: [num_states, num_actions]).
        gamma (float): Discount factor.
        rho (ndarray): Initial state distribution.
        policy_inits (list of ndarray): List of initial policies (shape: [num_states, num_actions]).
        n_iter (int): Number of iterations for the policy gradient.

    Returns:
        None
    """
    with PdfPages(output_pdf) as pdf:
        # To store results for all combinations
        results = []

        # 颜色列表用于区分不同初始策略
        colors = ['blue', 'green', 'red', 'purple', 'orange']
        markers = ['o', 's', 'D', '^', 'v']  # 不同标记

        for alpha in alpha_values:
            for eta in eta_values:
                plt.figure(figsize=(8, 8))
                plt.title(f"Policy Trajectories (eta={eta}, alpha={alpha}, tau={tau})")

                # 对每个初始策略运行算法并绘图
                for idx, policy_init in enumerate(policy_inits):
                    # 运行算法
                    V_history, Q_history, discounted_rewards, policy_trajectory = projected_policy_gradient_interpolation(
                        states, actions, transition_probs, rewards, gamma, rho, policy_init, eta, n_iter, alpha, tau
                    )

                    # 提取坐标
                    x_coords = [policy[0, 0] for policy in policy_trajectory]
                    y_coords = [policy[1, 0] for policy in policy_trajectory]

                    # 生成策略标签（基于初始值）
                    init_label = f"Init: s0={policy_init[0,0]:.2f}, s1={policy_init[1,0]:.2f}"

                    # 绘制轨迹
                    plt.plot(x_coords, y_coords, '-', 
                            color=colors[idx % len(colors)],
                            marker=markers[idx % len(markers)],
                            markersize=4,
                            linewidth=1.5,
                            alpha=0.8,
                            label=init_label)

                    # 标记初始点和终点
                    plt.scatter(x_coords[0], y_coords[0], 
                               color=colors[idx % len(colors)],
                               s=100, 
                               edgecolor='black',
                               zorder=10)
                    plt.scatter(x_coords[-1], y_coords[-1],
                               color=colors[idx % len(colors)],
                               s=200,
                               marker='X',  # 用X标记终点
                               edgecolor='black',
                               zorder=10)

                # 统一设置图表属性
                plt.xlabel("π(s0, a0)")
                plt.ylabel("π(s1, a0)")
                plt.xlim(0, 1)
                plt.ylim(0, 1)
                plt.grid(True)
                plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # 图例放在右侧

                # 保存当前组合的图表
                pdf.savefig(bbox_inches='tight')
                plt.close()

        # 添加总结页
        plt.figure(figsize=(10, 6))
        plt.title("Summary of All Experiments", fontsize=14)
        plt.axis('off')
        
        summary_text = "Experiment Parameters:\n"
        summary_text += f"- States: {states}\n"
        summary_text += f"- Actions: {actions}\n"
        summary_text += f"- Gamma: {gamma}\n"
        summary_text += "\nInitial Policies:\n"
        for idx, policy in enumerate(policy_inits):
            summary_text += f"Policy {idx+1}:\n{np.round(policy, 2)}\n\n"
        
        plt.text(0.1, 0.5, summary_text, ha='left', va='center', fontsize=10)
        pdf.savefig()
        plt.close()


### 下面直接验证论文中的MDP反例1

In [66]:
# 修改后的主程序部分
if __name__ == "__main__":
    # MDP参数保持不变
    states = [0, 1, 2]
    actions = [0, 1]
    transition_probs = np.array([
        [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]],
        [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0]],
        [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
    ])
    rewards = np.array([
        [-1, -1],
        [-1.6, -1.6],
        [0, 0]
    ])
    gamma = 0.5
    rho = np.array([1.0, 0.0, 0.0])

    # 定义多个不同的初始策略
    policy_inits = [
        # 初始策略1 (原策略)
        np.array([
            [0.96, 0.04],
            [0.92, 0.08],
            [0.50, 0.50]
        ]),
        # 初始策略2 (更均匀的分布)
        np.array([
            [0.98, 0.02],
            [0.80, 0.20],
            [0.50, 0.50]
        ]),
        # 初始策略3 (偏向另一个动作)
        np.array([
            [0.99, 0.01],
            [0.70, 0.30],
            [0.50, 0.50]
        ])
    ]

    n_iter = 200000
    eta_values = [0.01]
    alpha_values = [0.5]
    tau = 0

    # 输出PDF文件
    output_pdf = "multi_init_policy_trajectories_0.pdf"

    # 调用修改后的函数
    save_policy_trajectories_to_pdf(
        output_pdf=output_pdf,
        eta_values=eta_values,
        alpha_values=alpha_values,
        states=states,
        actions=actions,
        transition_probs=transition_probs,
        rewards=rewards,
        gamma=gamma,
        rho=rho,
        policy_inits=policy_inits,
        n_iter=n_iter,
        tau=tau
    )

    print(f"All policy trajectories and results have been saved to {output_pdf}.")

Termination conditions met.
Termination conditions met.
Termination conditions met.
All policy trajectories and results have been saved to multi_init_policy_trajectories_0.pdf.
